# Variables

In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


# Setup

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': '689WEDWQ67PBGCB2',
  'HostId': 'HXQCZFoh+H+/ZQmgHu/c3fawzLM+o1/mX9sWKdF2DHhm7vOQuHA8UmOtvqSMDR5P/YxPE+2v48g=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'HXQCZFoh+H+/ZQmgHu/c3fawzLM+o1/mX9sWKdF2DHhm7vOQuHA8UmOtvqSMDR5P/YxPE+2v48g=',
   'x-amz-request-id': '689WEDWQ67PBGCB2',
   'date': 'Sun, 30 Mar 2025 04:10:48 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-dev-sandbox',
   'CreationDate': datetime.datetime(2025, 2, 7, 23, 18, 12, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'era5-bra

In [5]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=8,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-03-29 23:10:50,294][INFO    ][coiled] Fetching latest package priorities...
[2025-03-29 23:10:50,295][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-03-29 23:10:51,158][INFO    ][coiled.package_sync] Scanning 444 conda packages...
[2025-03-29 23:10:51,166][INFO    ][coiled.package_sync] Scanning 259 python packages...
[2025-03-29 23:10:52,173][INFO    ][coiled] Running pip check...
[2025-03-29 23:10:54,158][INFO    ][coiled] Validating environment...
[2025-03-29 23:10:56,359][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-03-29 23:10:56,672][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-03-29 23:10:56,682][INFO    ][coiled] Uploading coiled_local_identifyingLandSubdivisions...
[2025-03-29 23:10:57,613][INFO    ][coiled] Requesti

Started a new Dask client on Coiled. Dashboard is available at https://cluster-mceib.dask.host/s-8rZ2K9so1ysToQ/status


# RUN

In [8]:
import s3fs
import fsspec
import traceback

import os


fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]
len(cities)

406

In [11]:
import numpy as np
import pandas as pd
import geopandas as gpd
import math
import dask.dataframe as dd
import dask_geopandas as dgpd
from dask import delayed, compute, visualize
from citywide_calculation import get_utm_crs

@delayed
def get_epsg(city_name):
    search_buffer = f'{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet'
    extent = dgpd.read_parquet(search_buffer)
    geometry = extent.geometry[0].compute()
    epsg = get_utm_crs(geometry)
    print(f'{city_name} EPSG: {epsg}')
    return epsg

def load_dataset(path, epsg=None):
    dataset = dgpd.read_parquet(path, npartitions=4)
    
    # Only assign if the file has no CRS
    if epsg:
        if dataset.crs is None:
            dataset = dataset.set_crs("EPSG:4326")  # assume WGS84 if missing
        dataset = dataset.to_crs(epsg)

    return dataset

In [20]:


# --- Step 1: Compute Bearings Vectorized ---

def compute_bearing_vectorized(x1, y1, x2, y2):
    """Compute bearing (in degrees) from (x1,y1) to (x2,y2) in vectorized form."""
    # Compute differences
    dx = x2 - x1
    dy = y2 - y1
    angles_rad = np.arctan2(dy, dx)
    angles_deg = np.degrees(angles_rad) % 360
    return angles_deg

def compute_intersection_angles(roads_df, intersections_df):
    """
    For each road, compute the bearing at the intersection.
    
    For each road, we assume:
      - If an intersection is the start (u), we compute the bearing from that intersection
        (using its coordinates from intersections_df) to the road's centroid.
      - Similarly for the end (v).
    
    Returns a DataFrame with columns:
      intersection_id, bearing
    """
    # Merge for start intersections
    roads_u = roads_df.merge(intersections_df[['osmid', 'geometry']], left_on='u', right_on='osmid', how='left', suffixes=('', '_u'))
    # Extract coordinates: intersection (start) and road centroid
    roads_u['x_u'] = roads_u.geometry_u.x
    roads_u['y_u'] = roads_u.geometry_u.y
    roads_u['centroid_x'] = roads_u.geometry.centroid.x
    roads_u['centroid_y'] = roads_u.geometry.centroid.y
    roads_u['bearing'] = compute_bearing_vectorized(roads_u['x_u'], roads_u['y_u'],
                                                      roads_u['centroid_x'], roads_u['centroid_y'])
    roads_u = roads_u[['u', 'bearing']].rename(columns={'u': 'intersection_id'})
    
    # Merge for end intersections
    roads_v = roads_df.merge(intersections_df[['osmid', 'geometry']], left_on='v', right_on='osmid', how='left', suffixes=('', '_v'))
    roads_v['x_v'] = roads_v.geometry_v.x
    roads_v['y_v'] = roads_v.geometry_v.y
    roads_v['centroid_x'] = roads_v.geometry.centroid.x
    roads_v['centroid_y'] = roads_v.geometry.centroid.y
    roads_v['bearing'] = compute_bearing_vectorized(roads_v['x_v'], roads_v['y_v'],
                                                      roads_v['centroid_x'], roads_v['centroid_y'])
    roads_v = roads_v[['v', 'bearing']].rename(columns={'v': 'intersection_id'})
    
    # Combine both: Each row corresponds to a road connected to an intersection, with its computed bearing.
    intersection_angles = dd.concat([roads_u, roads_v], interleave_partitions=True)
    return intersection_angles

# --- Step 2: Compute Sequential Differences for Each Intersection ---


def compute_sequential_differences(angles):
    """
    Given a sorted array of angles (in degrees), compute the circular differences
    between consecutive angles (including the wrap-around).
    """
    # Append the first angle plus 360 to account for wrap-around
    extended = np.concatenate([angles, [angles[0] + 360]])
    return np.diff(extended)

def compute_intersection_metric(group, street_count_mapping):
    """
    Computes the metric for one intersection.
    
    Parameters:
      group: DataFrame group (with a column 'bearing') for a given intersection.
      street_count_mapping: dict mapping intersection_id to its street_count.
      
    Process:
      1. Get the unique bearings (the "true" angles).
      2. Use the provided street_count (if available) to confirm the number of unique angles.
      3. Sort the unique angles and compute the circular differences.
      4. For 3‑way intersections (street_count==3): select the smallest difference.
         For 4‑way intersections (street_count==4): select the two smallest differences and average them.
      5. Compute the absolute difference between the selected value(s) and 90°.
    """
    inter_id = group.name  # group name is the intersection_id
    # Get expected street count for this intersection
    expected_sc = street_count_mapping.get(inter_id, None)
    
    # Deduplicate angles
    unique_angles = np.unique(group['bearing'].values)
    sc = len(unique_angles)
    
    # If we have an expected street count, use that if possible.
    # (It might differ if data are noisy.)
    if expected_sc is not None and expected_sc in (3, 4):
        sc = expected_sc
    else:
        # Only process intersections with 3 or 4 unique angles
        if sc not in (3, 4):
            return np.nan

    # Ensure we have exactly sc unique angles
    angles = np.sort(unique_angles)
    
    # Compute circular differences
    diffs = compute_sequential_differences(angles)
    
    if sc == 3:
        # For a 3-way, select the smallest difference
        selected_diff = np.min(diffs)
        metric = abs(90 - selected_diff)
    elif sc == 4:
        # For a 4-way, select the two smallest differences and average the absolute differences from 90°
        selected_diffs = np.sort(diffs)[:2]
        metric = np.mean(np.abs(90 - selected_diffs))
    else:
        metric = np.nan
    
    return metric


def compute_intersection_mapping(intersection_angles, street_count_mapping):
    """
    Given a DataFrame 'intersection_angles' with columns ['intersection_id', 'bearing'],
    group by intersection_id and compute the metric using the known street counts.
    Returns a Series mapping intersection_id to the computed metric.
    """
    # Group by intersection_id and apply the metric function.
    mapping = intersection_angles.groupby('intersection_id').apply(
        lambda grp: compute_intersection_metric(grp, street_count_mapping)
    )
    return mapping


def calculate_tortuosity(roads_df, intersections_df):

    # Merge roads with intersections for start point (u)
    roads_with_start = roads_df.merge(
        intersections_df[['osmid', 'geometry']],
        left_on='u',
        right_on='osmid',
        how='left',
        suffixes=('', '_start')
    )

    # Merge with intersections again for end point (v)
    roads_with_both = roads_with_start.merge(
        intersections_df[['osmid', 'geometry']],
        left_on='v',
        right_on='osmid',
        how='left',
        suffixes=('', '_end')
    )

    # Rename the merged geometry columns for clarity
    roads_with_both = roads_with_both.rename(
        columns={'geometry': 'road_geometry',
                'geometry_start': 'start_geom',
                'geometry_end': 'end_geom'}
    )

    # --- Step 2: Compute Straight-Line Distance between Intersections ---

    # Use the .distance method from GeoPandas (this assumes a projected CRS)
    roads_with_both['straight_distance'] = roads_with_both.apply(
        lambda row: row['start_geom'].distance(row['end_geom']), axis=1
    )

    # --- Step 3: Compute Road Length (if not already present) ---

    if 'length' not in roads_with_both.columns:
        roads_with_both['length'] = roads_with_both['road_geometry'].length

    # --- Step 4: Compute Tortuosity ---
    # Tortuosity is defined as road_length divided by the straight-line distance.
    # To avoid division by zero, we use np.where to set tortuosity to NaN when straight_distance is 0.

    roads_with_both['tortuosity'] = np.where(
        roads_with_both['straight_distance'] > 0,
        roads_with_both['straight_distance']/ roads_with_both['length'],
        np.nan
    )
    
    return roads_with_both


def metrics_roads_intersections(city_name):

    paths = {
    'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
    'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
    'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
    'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }

    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg)
    roads = load_dataset(paths['roads'], epsg=epsg)
    intersections = load_dataset(paths['intersections'], epsg=epsg).compute()

    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])

    intersections['osmid'] = intersections['osmid'].astype(int)
    intersection_angles = compute_intersection_angles(roads, intersections)
    street_count_mapping = intersections.set_index('osmid')['street_count'].to_dict()
    intersection_angle_mapping = compute_intersection_mapping(intersection_angles, street_count_mapping)
    intersection_angle_mapping = intersection_angle_mapping.compute()  


    intersections_with_angles_metric = intersections.merge(
        intersection_angle_mapping.rename("average_angle"), left_on="osmid", right_index=True, how="left"
    )

    joined_intersection_angles_grid = dgpd.sjoin(intersections_with_angles_metric, grid, predicate="within")
    average_angle_between_roads = joined_intersection_angles_grid.groupby('index_right')['average_angle'].mean()


    roads_with_tortuosity = calculate_tortuosity(roads.compute(), intersections)
    joined_tortuosity_grid = dgpd.sjoin(roads_with_tortuosity.set_geometry('road_geometry'), grid, predicate="within")
    average_tortuosity = joined_tortuosity_grid.groupby('index_right')['tortuosity'].mean()


    grid['metric_9'] = grid.index.map(average_tortuosity).fillna(-999.).astype(float)
    grid['metric_10'] = grid.index.map(average_angle_between_roads).fillna(-999.).astype(float)

    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_metrics_9_10_{YOUR_NAME}.geoparquet'

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')

    grid.to_parquet(path)

    return path

# --- Example Usage ---

import time
start_time = time.time()  

cities = ['Nairobi']

tasks = [delayed(metrics_roads_intersections)(city) for city in cities]

results = compute(*tasks)

end_time = time.time()  
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")






#print(mapping.describe())


Tasks completed in 21.91 seconds.


In [21]:
city_name = 'Nairobi'

paths = {
'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
}

epsg = get_epsg(city_name).compute()
grid = load_dataset(paths['grid'], epsg=epsg)
roads = load_dataset(paths['roads'], epsg=epsg)
intersections = load_dataset(paths['intersections'], epsg=epsg).compute()

if 'geom' in grid.columns:
    grid = grid.drop(columns=['geom'])

intersections['osmid'] = intersections['osmid'].astype(int)
intersection_angles = compute_intersection_angles(roads, intersections)
street_count_mapping = intersections.set_index('osmid')['street_count'].to_dict()
intersection_angle_mapping = compute_intersection_mapping(intersection_angles, street_count_mapping)
intersection_angle_mapping = intersection_angle_mapping.compute()  


intersections_with_angles_metric = intersections.merge(
    intersection_angle_mapping.rename("average_angle"), left_on="osmid", right_index=True, how="left"
)

joined_intersection_angles_grid = dgpd.sjoin(intersections_with_angles_metric, grid, predicate="within")
average_angle_between_roads = joined_intersection_angles_grid.groupby('index_right')['average_angle'].mean()


/var/folders/mt/3n9j2kc92kv4psztx687vtd80000gn/T/ipykernel_76178/2723016076.py:121: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  mapping = intersection_angles.groupby('intersection_id').apply(


In [25]:
intersections_with_angles_metric.to_parquet('intersections_with_angles.geoparquet')

In [ ]:
intersections_df['osmid'] = intersections_df['osmid'].astype('int')

def calculate_tortuosity(roads_df, intersections_df):

    # Merge roads with intersections for start point (u)
    roads_with_start = roads_df.merge(
        intersections_df[['osmid', 'geometry']],
        left_on='u',
        right_on='osmid',
        how='left',
        suffixes=('', '_start')
    )

    # Merge with intersections again for end point (v)
    roads_with_both = roads_with_start.merge(
        intersections_df[['osmid', 'geometry']],
        left_on='v',
        right_on='osmid',
        how='left',
        suffixes=('', '_end')
    )

    # Rename the merged geometry columns for clarity
    roads_with_both = roads_with_both.rename(
        columns={'geometry': 'road_geometry',
                'geometry_start': 'start_geom',
                'geometry_end': 'end_geom'}
    )

    # --- Step 2: Compute Straight-Line Distance between Intersections ---

    # Use the .distance method from GeoPandas (this assumes a projected CRS)
    roads_with_both['straight_distance'] = roads_with_both.apply(
        lambda row: row['start_geom'].distance(row['end_geom']), axis=1
    )

    # --- Step 3: Compute Road Length (if not already present) ---

    if 'length' not in roads_with_both.columns:
        roads_with_both['length'] = roads_with_both['road_geometry'].length

    # --- Step 4: Compute Tortuosity ---
    # Tortuosity is defined as road_length divided by the straight-line distance.
    # To avoid division by zero, we use np.where to set tortuosity to NaN when straight_distance is 0.

    roads_with_both['tortuosity'] = np.where(
        roads_with_both['straight_distance'] > 0,
        roads_with_both['straight_distance']/ roads_with_both['length'],
        np.nan
    )
    
    return roads_with_both

# --- Optionally: Check Summary Statistics ---
print(roads_with_both['tortuosity'].describe())

# The resulting roads_with_both GeoDataFrame now has a 'tortuosity' column.

count    320224.000000
mean          0.965383
std           0.092962
min           0.007647
25%           0.991078
50%           0.996044
75%           0.999557
max           1.001610
Name: tortuosity, dtype: float64


In [15]:
roads_with_both.columns

NameError: name 'roads_with_both' is not defined

In [26]:
client.close()